### Connection configuration

#### Library imports

In [129]:
import json
import time
import logging

from collections import namedtuple
# Cannot detect imported library from source (still working)
from requests import Session  # type: ignore
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects # type: ignore

#### API constants

In [130]:
KEY = "bffb27c6-a678-41df-8e4a-c3de6a590856"
START = 1
LIMIT = 10
CONVERT = "USD"
JSON = "application/json"
TYPE = "sandbox"
HOUR = "1h"
DAY = "24h"
WEEK = "7d"
MONTH = "30d"
SEPARATOR = "~*~"

#### API libraries, parsing libraries and parsing utilities

In [131]:
def decode(dict_obj: dict) -> object:
  try:
    return namedtuple("obj", dict_obj.keys())(*dict_obj.values())
  
  except Exception:
    return None

#### Session management

In [132]:
try:   
  session = Session()
  headers = {
    "Accepts": JSON,
    "X-CMC_PRO_API_KEY": KEY,
  }
  session.headers.update(headers)

  if session is None:
    raise Exception()
  
except Exception as error:
  print("⚠️ Failed to create session \n")
  logging.error(error)

### Data schema and presentation

#### Currency

In [133]:
class Currency:
    def __init__(self, quote_list: list, change_list: list, metadata_list: list):
        self.quote_list = quote_list
        self.change_list = change_list
        self.metadata_list = metadata_list

    def print_metadata(self):    
        """
        Print metadata of the top cryptocurrencies
        """

        j = 0

        for i in self.metadata_list: 
            hour_change = self.change_list[j][HOUR]
            day_change = self.change_list[j][DAY]
            week_change = self.change_list[j][WEEK]

            print(f"{i["name"]} ({i["symbol"]}) is ranked at position {i["rank"]}.")
            print(f"1h price change: {hour_change}%") 
            print(f"24h price change: {day_change}%") 
            print(f"7d price change: {week_change}%")
            print("\n")

            j += 1

    def get_metadata(self, currency: object) -> list:
        """"
        Get metadata of the top cryptocurrencies
        """

        metadata = {
            "id": currency.id,
            "name": currency.name,
            "symbol": "$" + currency.symbol,
            "rank": currency.cmc_rank,
        }
        self.metadata_list.append(metadata) 

    def get_change(self, currency: object) -> list:
        """
        Get price change of the top cryptocurrencies
        """

        change = {
            "id": currency.id,
            HOUR: round(currency.quote.USD.percent_change_1h, 2),
            DAY: round(currency.quote.USD.percent_change_24h, 2),
            WEEK: round(currency.quote.USD.percent_change_7d, 2),
        }
        self.change_list.append(change) 

    def get_quote(self, currency: object) -> list:
        """
        Get price of the top cryptocurrencies
        """

        quote = {
            "id": currency.id,
            "price": round(currency.quote.USD.price, 2)
        }
        self.quote_list.append(quote) 

#### Exchange

In [134]:
class Exchange:
    def __init__(self, quote_list: list, metadata_list: list):
        self.quote_list = quote_list
        self.metadata_list = metadata_list

    def get_quote(self, exchange: object) -> list:
        """
        Get quote of the top exchanges
        """

        quote = {
            "volume": {
                DAY: round(exchange.quote.USD.volume_24h, 2),
                WEEK: round(exchange.quote.USD.volume_7d, 2),
                MONTH: round(exchange.quote.USD.volume_30d, 2),
            },
            "percent_change": {
                DAY: round(exchange.quote.USD.percent_change_volume_24h, 2),
                WEEK: round(exchange.quote.USD.percent_change_volume_7d, 2),
                MONTH: round(exchange.quote.USD.percent_change_volume_30d, 2),
            },
            "effective_liquidity": {
                DAY: round(exchange.quote.USD.effective_liquidity_24h, 2)
            }
        }
        self.quote_list.append(quote)

    def get_metadata(self, exchange: object) -> list:
        """
        Get metadata of the top exchanges
        """

        metadata = {
            "id": exchange.id,
            "name": exchange.name,
            "rank": exchange.rank,
        }
        self.metadata_list.append(metadata)

    def print_metadata(self):
        """
        Print metadata of the top exchanges
        """
        
        j = 0

        for i in self.metadata_list:
            day_volume = self.quote_list[j]["volume"][DAY]
            week_volume = self.quote_list[j]["volume"][WEEK]
            month_volume = self.quote_list[j]["volume"][MONTH]
            day_change = self.quote_list[j]["percent_change"][DAY]
            week_change = self.quote_list[j]["percent_change"][WEEK]
            month_change = self.quote_list[j]["percent_change"][MONTH]
            liquidity = self.quote_list[j]["effective_liquidity"][DAY]

            print(f"{i["name"]} is ranked at position {i["rank"]}.")
            print(f"24h exchange volume: {day_volume}$")
            print(f"7d exchange volume: {week_volume}$")
            print(f"30d exchange volume: {month_volume}$")
            print(f"24h exchange volume change: {day_change}%")
            print(f"7d exchange volume change: {week_change}%")
            print(f"30d exchange volume change: {month_change}%")
            print(f"24h effective liquidity: {liquidity}$")
            print("\n")

            j += 1

### Data retrieval

#### Data fetch

In [135]:
def get_crypto_listings(dataset: object, verbose: bool):
    """
    Get cryptocurrency listings
    """
    try:
        cryptocurrency = Currency([], [], [])

        for crypto in range(0, LIMIT):
            Currency.get_metadata(currency=dataset[crypto], self=cryptocurrency)
            Currency.get_change(currency=dataset[crypto], self=cryptocurrency)
            Currency.get_quote(currency=dataset[crypto], self=cryptocurrency)
            
        if verbose:
            Currency.print_metadata(self=cryptocurrency)

    except Exception as error:
        print("⚠️ Failed to retrieve cryptocurrency listings.")
        logging.error(error)

def get_exchange_listings(dataset: object, verbose: bool):
    """
    Get exchange listings
    """

    try:
        cryptocurrency_exchange = Exchange([], [])

        for exchange in range(0, LIMIT):
            Exchange.get_metadata(exchange=dataset[exchange], self=cryptocurrency_exchange)
            Exchange.get_quote(exchange=dataset[exchange], self=cryptocurrency_exchange)

        if verbose:
            Exchange.print_metadata(self=cryptocurrency_exchange)

    except Exception as error:
        print("⚠️ Failed to retrieve exchange listings.")
        logging.error(error)

def get_global_metrics(dataset: object):
    """
    Get global metrics
    """
    
    try:
        active_cryptocurrencies = dataset.active_cryptocurrencies
        total_cryptocurrencies = dataset.total_cryptocurrencies
        active_exchanges = dataset.active_exchanges
        total_exchanges = dataset.total_exchanges

        print(f"Bitcoin dominance figure is currently at {round(dataset.btc_dominance, 2)}%.")
        print(f"Ethereum dominance figure is currently at {round(dataset.eth_dominance, 2)}%.")
        print(f"There are {active_cryptocurrencies} active cryptocurrencies out of {total_cryptocurrencies}.")
        print(f"There are {active_exchanges} active exchanges out of {total_exchanges}.\n")

    except Exception as error:
        print("⚠️ Failed to retrieve global metrics.")
        logging.error(error)

#### API call

In [136]:
def retrieve(path: str, parameters: object):
    """
    Retrieve data from API
    """
    
    try:    
        url = f"https://{TYPE}-api.coinmarketcap.com/v1/{path}"
        response = session.get(url, params=parameters)
        data = json.loads(response.text, object_hook=decode)

        return data

    except Exception as error:
        print("⚠️ Failed to retrieve data from API.")
        logging.error(error)
        return

def call(path: str, parameters: object):
    """
    Call API
    """

    result = retrieve(path=path, parameters=parameters)

    try:
        if path == "cryptocurrency/listings/latest":
            get_crypto_listings(dataset=result.data, verbose=True)

        elif path == "exchange/listings/latest":
            get_exchange_listings(dataset=result.data, verbose=True)

        elif path == "global-metrics/quotes/latest":
            get_global_metrics(dataset=result.data)

    except Exception:
        print(result.data.status.error_message, "\n")


### User input

#### User interface

In [137]:
def display_menu():
    """
    Display menu
    """
    
    options = [
        "Get cryptocurrency listings",
        "Get exchange listings",
        "Get global metrics",
        "Exit"
    ]

    print(SEPARATOR * 6, "MENU", SEPARATOR * 6)

    for i in range(1, len(options) + 1):
        print(f"{i}. {options[i - 1]}")
    
    print(SEPARATOR * 14, "\n")

#### Execution

In [138]:
def __main__(choices: dict):
  try:    
    while True:
      display_menu()
      choice = input("Enter your choice: ")
      
      if choice == "1":
        call(path=choices["1"]["path"], parameters=choices["1"]["parameters"])
      elif choice == "2":
        call(path=choices["2"]["path"], parameters=choices["2"]["parameters"])
      elif choice == "3":
        call(path=choices["3"]["path"], parameters=choices["3"]["parameters"])
      elif choice == "4":
        break
      else:
        print("Invalid choice. Please try again. \n")

  except (ConnectionError, Timeout, TooManyRedirects) as error:
    print("⚠️ Failed to connect to the API.")
    logging.error(error)

### Entry point

In [139]:
choices = {
    "1": {
        "path": "cryptocurrency/listings/latest",
        "parameters": { "limit": str(LIMIT), "convert": CONVERT }
    },
    "2": {
        "path": "exchange/listings/latest",
        "parameters": { "limit": str(LIMIT), "convert": CONVERT }
    },
    "3": {
        "path": "global-metrics/quotes/latest",
        "parameters": { "convert": CONVERT }
    }
}

try:
    start = time.time()
    __main__(choices=choices)
    end = time.time()
    
    execution_time = round(end - start, 2)
    print(f"Program ran in {execution_time}s  .")

except Exception as error:
    print("⚠️ Failed to run program.")
    logging.error(error)

~*~~*~~*~~*~~*~~*~ MENU ~*~~*~~*~~*~~*~~*~
1. Get cryptocurrency listings
2. Get exchange listings
3. Get global metrics
4. Exit
~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~ 

llb7d2da4da ($wr0vgyvsh7o) is ranked at position 8656.
1h price change: 0.56%
24h price change: 0.61%
7d price change: 0.52%


muh4asgxzb ($i7pif0ovxf) is ranked at position 7152.
1h price change: 0.5%
24h price change: 0.2%
7d price change: 0.93%


ny6dqvhp3qj ($k1zukkd3cd) is ranked at position 9730.
1h price change: 0.62%
24h price change: 0.55%
7d price change: 0.57%


llfl7bkj7c8 ($4nfcifv3yaw) is ranked at position 6569.
1h price change: 0.39%
24h price change: 0.8%
7d price change: 0.44%


5gardq1r0ml ($21xxu3x3718) is ranked at position 2847.
1h price change: 0.37%
24h price change: 0.87%
7d price change: 0.75%


iy4ivsjnd9r ($dctm4s8osnc) is ranked at position 2141.
1h price change: 0.12%
24h price change: 0.04%
7d price change: 0.63%


vrivmw6vl5 ($ycvd6awuee) is ranked at position 5978.
1h price change: 